In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.neighbors import KNeighborsClassifier

# Charger les données
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

# Préparer les données
# Supprimer des colonnes inutiles
columns_to_drop = ['Name', 'Ticket', 'Cabin']  # Peu utiles pour le KNN
train_data = train_data.drop(columns=columns_to_drop)
test_data = test_data.drop(columns=columns_to_drop)

# Gérer les valeurs manquantes
train_data['Age'].fillna(train_data['Age'].median(), inplace=True)
test_data['Age'].fillna(test_data['Age'].median(), inplace=True)
test_data['Fare'].fillna(test_data['Fare'].median(), inplace=True)

# Encoder les colonnes catégoriques
encoder = LabelEncoder()
train_data['Sex'] = encoder.fit_transform(train_data['Sex'])
test_data['Sex'] = encoder.transform(test_data['Sex'])

train_data['Embarked'] = encoder.fit_transform(train_data['Embarked'].astype(str))
test_data['Embarked'] = encoder.transform(test_data['Embarked'].astype(str))

# Séparer les features et la cible
X = train_data.drop(columns=['Survived', 'PassengerId'])
y = train_data['Survived']
X_test = test_data.drop(columns=['PassengerId'])
passenger_ids = test_data['PassengerId']

# Normaliser les données
scaler = StandardScaler()
X = scaler.fit_transform(X)
X_test = scaler.transform(X_test)

# Diviser les données d'entraînement pour validation
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Entraîner le modèle KNN
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)

# Évaluer sur les données de validation
score = knn.score(X_val, y_val)
print(f'Accuracy on validation set: {score:.2f}')

# Prédire sur les données de test
predictions = knn.predict(X_test)

# Sauvegarder les prédictions
output = pd.DataFrame({
    'PassengerId': passenger_ids,
    'Survived': predictions
})
output.to_csv('submission.csv', index=False)
print("Fichier submission.csv créé avec succès.")
